In [1]:
from Acesso import Login
from Moeda import Moeda
from Query import Query
from RemoverArquivo import Remover
from WebTransfer import Web
import telebot
from telebot import types
from datetime import datetime
from pathlib import Path
import shutil
import os
import pandas as pd
from decouple import config
from glob import glob
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import pyttsx3

TOKEN=config('TOKEN')

bot=telebot.TeleBot(TOKEN,parse_mode='html')

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)


querys={

    'Canais':

    """
        
    SELECT e.[ID Cliente],e.SKU,p.Produto,p.Fabricante,p.Seção,p.Categoria,p.Linha,s.Segmento,s.Canal,
    SUM(e.[Total Venda]) AS [Total Venda]
    FROM netfeira.vw_targetestatistico e
    INNER JOIN netfeira.vw_produto p ON e.SKU=p.SKU AND p.Status='ATIVO'
    INNER JOIN netfeira.vw_cliente c ON e.[ID Cliente]=c.[ID Cliente] AND c.[Status do Cliente]='ATIVO'
    INNER JOIN netfeira.vw_segmento s ON c.[ID Segmento]=s.[ID Segmento]
    WHERE [Data de Faturamento] BETWEEN DATEADD(DAY,-90,DATEADD(DAY,-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)))
    AND DATEADD(DAY,-1,CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)) AND [Tipo de Operação]='VENDAS' AND [ID Situação] IN('FA','AB')
    GROUP BY e.[ID Cliente],e.SKU,p.Produto,p.Fabricante,p.Seção,p.Categoria,p.Linha,s.Segmento,s.Canal
        
    """

}

In [2]:
df=sql.CriarTabela(kwargs=querys)

c:\Users\eduardo\OneDrive - NETFEIRA PONTOCOM LTDA\Python\Homologação\Telegram BOT\Comercial\Query\sql_query.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabelas_df[tabela]=pd.read_sql(query,self.conectando)


In [3]:
df['Canais'].columns

Index(['ID Cliente', 'SKU', 'Produto', 'Fabricante', 'Seção', 'Categoria',
       'Linha', 'Segmento', 'Canal', 'Total Venda'],
      dtype='object')

In [5]:
def Classificacao(val):

    if val<=33.33:

        tipo='A'

        pass

    elif val<=66.66:

        tipo='B'

        pass


    else:

        tipo='C'

        pass


    return tipo


    pass

In [8]:
colunas=['Fabricante','Seção','Categoria','Linha','Produto']

tabela='Linha ABC'

col='Total Venda'

df['Temp']=df['Canais'].loc[df['Canais']['Segmento']=='RESTAURANTE']

for i,c in enumerate(colunas):
    
    temp_df=pd.DataFrame()
    
    if i==0:
            
        df[tabela]=df['Temp'].groupby(c,as_index=False).agg({col:'sum'})
        
        df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

        df[tabela]['Total']=df[tabela][col].sum()

        df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100

        valores=[]

        total=0

        for j in range(0,len(df[tabela])):

            total+=df[tabela].loc[j,col].sum()

            valores.append(total)

            pass

        df[tabela]['Acumulado']=valores

        df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

        df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

        df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
        
        pass
    
    else:
                
        indixe=df[tabela][colunas[i-1]].unique().tolist()
                
        for g in indixe:
        
            df[tabela]=df['Temp'].loc[df['Temp'][colunas[i-1]]==g].groupby(colunas[:i+1],as_index=False).agg({col:'sum'})

            df[tabela].sort_values(col,ascending=False,ignore_index=True,inplace=True)

            df[tabela]['Total']=df[tabela][col].sum()

            df[tabela]['Perc Geral']=round(df[tabela][col]/df[tabela]['Total'],4)*100        

            valores=[]

            total=0

            for j in range(0,len(df[tabela])):

                total+=df[tabela].loc[j,col].sum()

                valores.append(total)

                pass

            df[tabela]['Acumulado']=valores

            df[tabela]['Perc']=round(df[tabela]['Acumulado']/df[tabela]['Total'],4)*100

            df[tabela]['Classificação']=df[tabela]['Perc'].apply(Classificacao)

            df[tabela]=df[tabela].loc[df[tabela]['Classificação']!='C']
            
            temp_df=pd.concat([temp_df,df[tabela]],axis=0,ignore_index=True)
            
            pass
        
        df[tabela]=temp_df
                
        pass

    pass

df[tabela]

""


In [9]:
df['Canais'].loc[df['Canais']['Segmento']=='RESTAURANTE']

,ID Cliente,SKU,Produto,Fabricante,Seção,Categoria,Linha,Segmento,Canal,Total Venda
167,2027,12,POLPINHA DE ACEROLA (100G),DE MARCHI,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,RESTAURANTE,FS,487.96
168,2027,481,"CONGELADO VAGEM FRANCESA INT (2,5KG)",DAUCY,VEGETAIS CONGELADOS,VAGEM,"LEGUMES ATÉ 2,5KG",RESTAURANTE,FS,606.48
169,2027,484,CONGELADO MANDIOCA TOLETE DM 1.2KG,DE MARCHI,VEGETAIS CONGELADOS,MANDIOCA,"LEGUMES ATÉ 2,5KG",RESTAURANTE,FS,1848.20
170,2027,512,"CONGELADO ESPINAFRE FOLHA ( 2,5KG )",DAUCY,VEGETAIS CONGELADOS,ESPINAFRE FOLHA,"LEGUMES ATÉ 2,5KG",RESTAURANTE,FS,443.15
171,2027,651,CONGELADO ERVILHA DAUCY (2.5KG),DAUCY,VEGETAIS CONGELADOS,ERVILHA,"LEGUMES ATÉ 2,5KG",RESTAURANTE,FS,97.70
...,...,...,...,...,...,...,...,...,...,...
25505,29841,158,POLPINHA DE MORANGO (100G),DE MARCHI,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,RESTAURANTE,FS,73.98
25506,29841,1078,POLPINHA SUCO DE TANGERINA (100 GR),DE MARCHI,POLPAS DE FRUTAS,POLPINHA,POLPINHA 100G,RESTAURANTE,FS,60.09
25531,29850,9563,"CONG. PICKERS COXINHA MCCAIN CRU LINGUIÇA 1,05KG",MCCAIN,BATATAS CONGELADAS,APERITIVOS,APERITIVOS,RESTAURANTE,FS,70.88
25532,29850,9577,CONG. BATATA MCCAIN POPS CALABRESA 400G,MCCAIN,BATATAS CONGELADAS,BATATA ESPECIALIDADE,BATATA MASSA,RESTAURANTE,FS,64.80
